In [6]:

import torch
import torch
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.path.abspath(''), 'core'))
from fluxonium import Fluxonium
import general
import utils as utl



In [14]:
#Gradient Descent over T2 rate

#FIXED PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

dim = 110

learn_rate =1

#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)

EC.data = EC.data * (8 - 1e-3) + 1e-3
EJ.data = EJ.data * (20 - 2.5) + 2.5
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EL_bounds = [2e-1, 10]
EC_bounds = [1e-3, 8]

#RECORDING VALUES
EJ_vals = [EJ.item()]
EL_vals = [EL.item()]
EC_vals = [EC.item()]
T2_rate_results = [1e100, 1e99]

#we are trying to minise the rate

#GRADIENT DESCENT
i = 0
while T2_rate_results[-1] < T2_rate_results[-2]:
    i+=1

    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)
    EC = torch.tensor(EC.item(), requires_grad=True, dtype=torch.double)

    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, "sym_H")
    
    eigvals, eigvecs = fluxonium.esys()

    T2_rate  = general.t2_rate(qubit = fluxonium, eigvecs=eigvecs, eigvals =eigvals)

    T2_rate.backward()
    
    T2_rate_results.append(T2_rate.item())

    with torch.no_grad():
        #If paramater value hits boundary, it stays on boundary
        ##WHY DO WE minus the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EL_vals.append(EL.item())
    EC_vals.append(EL.item())

    print(f"{i}: R2: {T2_rate.item()}, EJ: {EJ.item()}, EL: {EL.item()}, EC: {EC.item()}")



1: R2: 0.001222315716103383, EJ: 4.491477786296765, EL: 2.663656122264877, EC: 2.511293249997444
2: R2: 0.0012214677934431824, EJ: 4.491475569333465, EL: 2.6645728736037095, EC: 2.511293249997444
3: R2: 0.0012206300995379468, EJ: 4.49147358996162, EL: 2.665483482147963, EC: 2.511293249997444
4: R2: 0.0012198040992560904, EJ: 4.491471846893126, EL: 2.6663867981771356, EC: 2.511293249997444
5: R2: 0.001218992038132212, EJ: 4.491470338621796, EL: 2.667281063213268, EC: 2.511293249997444
6: R2: 0.0012181973369324469, EJ: 4.491469063307468, EL: 2.668163536607059, EC: 2.511293249997444
7: R2: 0.0012174253088726346, EJ: 4.491468018589055, EL: 2.669029774036696, EC: 2.511293249997444
8: R2: 0.0012166845268705496, EJ: 4.491467201259007, EL: 2.6698721239780143, EC: 2.511293249997444
9: R2: 0.0012159895981053131, EJ: 4.491466606643911, EL: 2.670676272349416, EC: 2.511293249997444
10: R2: 0.0012153671691117606, EJ: 4.491466227284817, EL: 2.6714122964088194, EC: 2.511293249997444
11: R2: 0.00121486

In [13]:
#Gradient Descent Over Protection Factor


#FIXED PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

dim = 110

learn_rate =0.5

fluxonium = Fluxonium(EJ, EC, EL, flux, dim, "sym_H")
t1_noise_channels =  fluxonium.t1_supported_noise_channels()
tphi_noise_channels =  fluxonium.tphi_supported_noise_channels()


#VARIABLE PARAMS
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)

EC.data = EC.data * (8 - 1e-3) + 1e-3
EJ.data = EJ.data * (20 - 2.5) + 2.5
EL.data = EL.data * (10 - 2e-1) + 2e-1

EJ_bounds = [2.5, 150]
EL_bounds = [2e-1, 10]
EC_bounds = [1e-3, 8]

delta_EJ = ((EJ_bounds[1]-EJ_bounds[0])/2)*0.01
delta_EL = ((EL_bounds[1]-EL_bounds[0])/2)*0.01
delta_EC = ((EC_bounds[1]-EC_bounds[0])/2)*0.01

#RECORDING VALUES
EJ_vals = [EJ.item()]
EL_vals = [EL.item()]
EC_vals = [EC.item()]
protection_factor_results = [1e100, 1e99]

#we are trying to minise protection factor

#GRADIENT DESCENT
i = 0
for x in range(0,10):
    i+=1

    #Initiate qubit with current paramater values
    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)
    EC = torch.tensor(EC.item(), requires_grad=True, dtype=torch.double)

    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, "sym_H")
    
    eigvals, eigvecs = fluxonium.esys()

    T1_rate  = general.effective_t1_rate(qubit =fluxonium, eigvecs=eigvecs, eigvals =eigvals, noise_channels=t1_noise_channels)

    Tphi_rate  = general.effective_tphi_rate(qubit = fluxonium, eigvecs=eigvecs,  noise_channels=tphi_noise_channels)


    #Initiate qubit with current paramater values + delta
    EJ_delta = torch.tensor(EJ.item()+delta_EJ, requires_grad=True, dtype=torch.double)
    EL_delta = torch.tensor(EL.item()+delta_EL, requires_grad=True, dtype=torch.double)
    EC_delta = torch.tensor(EC.item()+delta_EC, requires_grad=True, dtype=torch.double)

    fluxonium_delta = Fluxonium(EJ_delta, EC_delta, EL_delta, flux, dim, "sym_H")
    
    eigvals, eigvecs = fluxonium.esys()

    T1_rate_delta  = general.effective_t1_rate(qubit =fluxonium, eigvecs=eigvecs, eigvals =eigvals, noise_channels=t1_noise_channels)

    Tphi_rate_delta  = general.effective_tphi_rate(qubit = fluxonium, eigvecs=eigvecs,  noise_channels=tphi_noise_channels)

    eigvals, eigvecs = fluxonium_delta.esys()

    T1_rate_delta  = general.effective_t1_rate(qubit =fluxonium_delta , eigvecs=eigvecs, eigvals =eigvals, noise_channels=t1_noise_channels)

    Tphi_rate_delta  = general.effective_tphi_rate(qubit =fluxonium_delta , eigvecs=eigvecs, noise_channels=tphi_noise_channels)

    #Calculate protection factor
    protection_factor = abs(0.5*( (T1_rate - T1_rate_delta)/(Tphi_rate - Tphi_rate_delta) + (Tphi_rate - Tphi_rate_delta)/(T1_rate - T1_rate_delta))) 

    #Calculate gradients and apply GD
    protection_factor.backward()
    
    protection_factor_results.append(protection_factor.item())

    with torch.no_grad():
        #If paramater value hits boundary, it stays on boundary
        ##WHY DO WE minus the gradient and not add?
        EJ.data = EJ - EJ.grad*learn_rate if EJ - EJ.grad*learn_rate > EJ_bounds[0] else torch.tensor(EJ_bounds[0])
        EL.data = EL - EL.grad*learn_rate if EL - EL.grad*learn_rate > EL_bounds[0] else torch.tensor(EL_bounds[0])
        EC.data = EC - EC.grad*learn_rate if EC - EC.grad*learn_rate > EC_bounds[0] else torch.tensor(EC_bounds[0])
        
        
    #RECORDING VALUES FOR PLOT
    EJ_vals.append(EJ.item())
    EC_vals.append(EC.item())
    EL_vals.append(EL.item())

    print(f"{i}: Protection Factor: {protection_factor.item()}, EJ: {EJ.item()}, EL: {EL.item()}, EC: {EC.item()}")



1: Protection Factor: 76.58759942359153, EJ: 13.643962292934193, EL: 657.4980480127886, EC: 0.0010000000474974513
2: Protection Factor: 42141172.87511427, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.599922
3: Protection Factor: 3.527377888277267, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.600506
4: Protection Factor: 3.5275465774843786, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.60109
5: Protection Factor: 3.5276867715022604, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.601673
6: Protection Factor: 3.527337370427876, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.602257
7: Protection Factor: 3.5274434035109654, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.60284
8: Protection Factor: 3.528135979404872, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.603424
9: Protection Factor: 3.5268782049733027, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.604008
10: Protection Factor: 3.527896474781875, EJ: 2.5, EL: 0.20000000298023224, EC: 9315743057.604591
